In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image

In [3]:
import torch
from torchvision import models

# Load VGG-16
vgg16 = models.vgg16(pretrained=False)
vgg16.classifier[6] = torch.nn.Linear(vgg16.classifier[6].in_features, 2)  # Modify the last layer
vgg16.load_state_dict(torch.load("vgg16_epoch_1.pth", map_location=torch.device('cpu')))
vgg16.eval()

# Load ResNet-18
resnet18 = models.resnet18(pretrained=False)
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, 2)  # Modify the last layer
resnet18.load_state_dict(torch.load("resnet18_1.pth", map_location=torch.device('cpu')))
resnet18.eval()

# Load ResNet-50
resnet50 = models.resnet50(pretrained=False)
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, 2)  # Modify the last layer
resnet50.load_state_dict(torch.load("resnet50_1.pth", map_location=torch.device('cpu')))
resnet50.eval()


c:\Users\Bhavya\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Bhavya\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Bhavya\AppData\Local\Temp\ipykernel_36012\4097387251.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fli

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Preprocessing: Transform for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

def predict(image, model_name):
    # Preprocess the image
    image = Image.fromarray(image)
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Select the model
    if model_name == "VGG-16":
        model = vgg16
    elif model_name == "ResNet-18":
        model = resnet18
    elif model_name == "ResNet-50":
        model = resnet50
    else:
        return "Invalid model selection!"

    # Run the model and get predictions
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)  # Get the class with max probability
        label = "Malignant" if predicted.item() == 1 else "Benign"

    return label


In [5]:
# Gradio interface
model_options = ["VGG-16", "ResNet-18", "ResNet-50"]

interface = gr.Interface(
    fn=predict,  # Prediction function
    inputs=[
        gr.components.Image(type="numpy", label="Input Image"),  # Image input
        gr.components.Dropdown(choices=model_options, label="Choose Model")  # Model selection
    ],
    outputs=gr.components.Textbox(label="Classification Result"),  # Output: Benign or Malignant
    title="TumorTrace: MRI-Based AI for Breast Cancer Detection",
    description="Upload an MRI-scan image and select a model to classify it as Benign or Malignant."
)

# Launch the interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
